In [18]:
import pandas as pd
import numpy as np

# Recreate the DataFrame since previous state was lost
regional_demand_TWh = {
    "DE0 2 heat": 2.237750e+08 / 1e6,
    "DE0 9 heat": 1.401345e+08 / 1e6,
    "DE0 A heat": 2.693371e+08 / 1e6,
    "DE0 E heat": 1.333434e+08 / 1e6
}

heating_shares = {
    "gas_burner": 0.55,
    "oil_burner": 0.19,
    "biomass_burner": 0.10,
}

legacy_techs = ["gas_burner", "oil_burner", "biomass_burner"]

full_lifetimes = {
    "gas_burner": 20,
    "oil_burner": 20,
    "biomass_burner": 25,

}

min_lifetime = {
    "gas_burner": 1,
    "oil_burner": 1,
    "biomass_burner": 1,

}

# Updated capacity factors (midpoints for ranges)
updated_capacity_factors = {
    "gas_burner": 0.4,
    "oil_burner": 0.4,
    "biomass_burner": 0.4,

}

entries = []

for bus, regional_TWh in regional_demand_TWh.items():
    for tech in legacy_techs:
        tech_share = heating_shares[tech]
        tech_demand_TWh = regional_TWh * tech_share
        n_units = int(np.ceil(tech_demand_TWh))
        lifetimes = np.linspace(min_lifetime[tech], full_lifetimes[tech], n_units, dtype=int)
        for i, life in enumerate(lifetimes):
            demand_TWh_per_unit = tech_demand_TWh / n_units
            p_nom = round(demand_TWh_per_unit * 1e6 / (8760 * updated_capacity_factors[tech]), 2)
            entries.append({
                "bus": bus,
                "technology": tech,
                "unit_id": i + 1,
                "demand_TWh": demand_TWh_per_unit,
                "remaining_lifetime": life,
                "decommissioning_year": 2020 + life,
                "p_nom": p_nom
            })

df = pd.DataFrame(entries)
# Step 3: Create 'carrier' column from 'technology'
df['carrier'] = df['technology'].str.extract(r'(\w+)_burner')

# Step 5: Add 'commissioning_year' using fixed lifetimes
df['commissioning_year'] = df.apply(
    lambda row: row['decommissioning_year'] - full_lifetimes.get(row['technology'], 0), axis=1)
df.to_csv('D:/project_h2/data/heating_techs/main_dummy_heating_techs.csv')

In [19]:
df

,bus,technology,unit_id,demand_TWh,remaining_lifetime,decommissioning_year,p_nom,carrier,commissioning_year
0,DE0 2 heat,gas_burner,1,0.992550,1,2021,283.26,gas,2001
1,DE0 2 heat,gas_burner,2,0.992550,1,2021,283.26,gas,2001
2,DE0 2 heat,gas_burner,3,0.992550,1,2021,283.26,gas,2001
3,DE0 2 heat,gas_burner,4,0.992550,1,2021,283.26,gas,2001
4,DE0 2 heat,gas_burner,5,0.992550,1,2021,283.26,gas,2001
...,...,...,...,...,...,...,...,...,...
647,DE0 E heat,biomass_burner,10,0.952453,17,2037,271.82,biomass,2012
648,DE0 E heat,biomass_burner,11,0.952453,19,2039,271.82,biomass,2014
649,DE0 E heat,biomass_burner,12,0.952453,21,2041,271.82,biomass,2016
650,DE0 E heat,biomass_burner,13,0.952453,23,2043,271.82,biomass,2018


creating 3 files from it, based on carrier

In [20]:


# Load the existing technologies file

# Extract the carrier from the 'tech' column (e.g., 'gas_burner' → 'gas')
df['carrier'] = df['technology'].apply(lambda x: x.split('_')[0])

# Save full version with 'carrier' column added (optional)
# df.to_csv("data/existing_heating_technologies_with_carrier.csv", index=False)

# Split the dataframe based on carrier type
df_gas = df[df['carrier'] == 'gas']
df_oil = df[df['carrier'] == 'oil']
df_biomass = df[df['carrier'] == 'biomass']

# Save each to a separate file
df_gas.to_csv("D:/project_h2/data/heating_techs/Gas_Heating_Technologies.csv", index=False)
df_oil.to_csv("D:/project_h2/data/heating_techs/Oil_Heating_Technologies.csv", index=False)
df_biomass.to_csv("D:/project_h2/data/heating_techs/Biomass_Heating_Technologies.csv", index=False)


In [21]:
df_gas

,bus,technology,unit_id,demand_TWh,remaining_lifetime,decommissioning_year,p_nom,carrier,commissioning_year
0,DE0 2 heat,gas_burner,1,0.992550,1,2021,283.26,gas,2001
1,DE0 2 heat,gas_burner,2,0.992550,1,2021,283.26,gas,2001
2,DE0 2 heat,gas_burner,3,0.992550,1,2021,283.26,gas,2001
3,DE0 2 heat,gas_burner,4,0.992550,1,2021,283.26,gas,2001
4,DE0 2 heat,gas_burner,5,0.992550,1,2021,283.26,gas,2001
...,...,...,...,...,...,...,...,...,...
607,DE0 E heat,gas_burner,70,0.991066,18,2038,282.84,gas,2018
608,DE0 E heat,gas_burner,71,0.991066,19,2039,282.84,gas,2019
609,DE0 E heat,gas_burner,72,0.991066,19,2039,282.84,gas,2019
610,DE0 E heat,gas_burner,73,0.991066,19,2039,282.84,gas,2019
